### Problem#1
Suppose you have table_1, table_2, table_3, table_4. You have to update table_4 column base on table_3. But there is a requirement that you can only access table_2, table_3, table_4 base on table_1.

#### Solution:##1
Here we will iterate through each table then update the table_4.
`PROBLEM` is that it is not an ideal solution  beacuse neasted for loops.

In [ ]:
do
$$
    declare
        pBook        record;
        pBookPage    record;
        pBookTextBox record;
    begin
        for pBook in select "ProBookId"
                     from "ProBooks"
                     where "IsSample" is true
            loop
                raise notice 'Book=% ', pBook."ProBookId";
                for pBookPage in select "ProBookPageId"
                                 from "ProBookPages"
                                 where "ProBookId" = pBook."ProBookId"
                    loop
                        raise notice 'Book=%  Pages=% ',pBook."ProBookId", pBookPage."ProBookPageId";
                        for pBookTextBox in select "ProBookTextBoxId"
                                            from "ProBookTextBoxes"
                                            where "ProBookPageId" = pBookPage."ProBookPageId"
                            loop
                                Update "ProBookTexts"
                                set "IsSample"= true
                                where "ProBookTextBoxId" = pBookTextBox."ProBookTextBoxId";
                            end loop;
                    end loop;
            end loop;
    end ;
$$;

#### Solution-2
1. Join table_1, table_2, table_3
2. Update table_4 base on table_3

In [ ]:
update "ProBookTexts"
        set "IsSample" = true
        where "ProBookTextBoxId" in (
            select tb."ProBookTextBoxId"
            from "ProBooks" b
                     join "ProBookPages" p on b."ProBookId" = p."ProBookId"
                     join "ProBookTextBoxes" tb on p."ProBookPageId" = tb."ProBookPageId"
            where b."IsSample" = true
        );

#### Solution-3: LINQ

```linq
allTextIds = context.ProBookTexts
        .Where(pbt => context.ProBooks
            .Where(b => b.IsSample)
            .Join(
                context.ProBookPages,
                b => b.ProBookId,
                p => p.ProBookId,
                (b, p) => new { b, p }
            )
            .Join(
                context.ProBookTextBoxes,
                bp => bp.p.ProBookPageId,
                tb => tb.ProBookPageId,
                (bp, tb) => tb.ProBookTextBoxId
            )
            .Contains(pbt.ProBookTextBoxId))
        .ToList();
```
or 
```
var textBoxIdsToUpdate = (from b in context.ProBooks
                            where b.IsSample
                              join p in context.ProBookPages on b.ProBookId equals p.ProBookId
                              join tb in context.ProBookTextBoxes on p.ProBookPageId equals tb.ProBookPageId
                              join t in context.ProBookTexts on tb.ProBookTextBoxId equals t.ProBookTextBoxId
                              select t.ProBookTextBoxId)
                            .ToList();
// then extract the texts
  var textsToUpdate = context.ProBookTexts
                            .Where(t => textBoxIdsToUpdate.Contains(t.ProBookTextBoxId))
                        .ToList();
```
or 
```
var textsToUpdate = context.ProBookTexts
    .Where(t => 
        (from b in context.ProBooks
        where b.IsSample
         join p in context.ProBookPages on b.ProBookId equals p.ProBookId
         join tb in context.ProBookTextBoxes on p.ProBookPageId equals tb.ProBookPageId
         select tb.ProBookTextBoxId)
         .Contains(t.ProBookTextBoxId))
    .ToList();
```

### More advanced SQL